In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/Project/aspectCategoryDetection.csv",delimiter='#',encoding = 'utf-8')
del categoryDF['aspectTermPolarity']

In [2]:
domain=[]
for id in categoryDF.index:
    domain.append(categoryDF.iloc[id,0][:3])
categoryDF['domain'] = domain
categoryDF.head()

,domainID,aspectCategoryPolarity,review,aspectCategory,domain
0,app_2,neu,फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की तरह य...,gui,app
1,app_3,neu,"इस पॉपअप बॉक्स में पासवर्ड चेंज करने, लॉगइन अल...",misc,app
2,app_4,neu,आप यहीं से पुराने लॉगइन सेशंस को हमेशा के लिए ...,gui,app
3,app_5,neu,ऐसे में कई बार ऐसा होता है कि आपका लॉगइन ओपन ह...,NaN,app
4,app_6,pos,अब नए फीचर से इस तरह की गलतियां नहीं होंगी और ...,NaN,app


In [3]:
categorySizeDF = pd.DataFrame({"count":categoryDF.groupby('domain',sort='False').size()})
categorySizeDF = categorySizeDF.reset_index()
categorySizeDF

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [4]:
temp = categorySizeDF.ix[10]
categorySizeDF.ix[10] = categorySizeDF.ix[11]
categorySizeDF.ix[11]= temp

In [5]:
categorySizeDF

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [6]:
dfList=[]
currentIndex= 0 
for id in categorySizeDF.index:    
    if id == 0:
        currentIndex=categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[:currentIndex])
        
    else:
        nextIndex= currentIndex  + categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[currentIndex:nextIndex])
        currentIndex=nextIndex



In [7]:
entertainmentDF = dfList[6].append(dfList[10])
electronicDF = dfList[1]
for i in range(len(dfList)):
    if i>1 and i!=6 and i!=10:
        electronicDF = electronicDF.append(dfList[i])

entertainmentDF = entertainmentDF.dropna(axis=0,how='any')
electronicDF = electronicDF.dropna(axis=0,how='any')

In [8]:
entertainmentDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8', index= False)
electronicDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8',index = False)

In [9]:
import codecs
import hindiTokenizer
from hindiTokenizer import Tokenizer
import nltk


with codecs.open("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8') as eReview:
    eReviewLines = eReview.readlines()

with codecs.open("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8') as elecReview:
    elecReviewLines = elecReview.readlines()


In [10]:
import codecs
reviewList=[]
posReview = codecs.open('Z:/MSBooks/NLP/Project/postaggedreviews.txt',encoding='utf-8')
reviewList=[x.strip() for x in posReview.readlines()]

In [11]:
print reviewList[1]

app_3/QC इस/DEM पॉपअप/NN बॉक्स/NN में/PSP पासवर्ड/JJ चेंज/NN करने/VM ,/SYM लॉगइन/NN अलर्ट्स/NNP चालू/NN करने/VM और/CC मौजूदा/JJ फेसबुक/NN सेशन/NN को/PSP पूरी/JJ तरह/NN से/PSP बंद/JJ करने/VM के/PSP ऑप्शन/NN मिलेंगे/VM ./SYM


In [12]:
from collections import OrderedDict
adjectives=OrderedDict()
for item in reviewList:
    idAndTagged = item.split()
    id = idAndTagged[0]
    taggedSentence = idAndTagged[1:]
    domain = id.split("/")[0]
    definingTerms = [s for s in taggedSentence if "JJ" in s]
    adjTerms = []
    for items in definingTerms:
        word=items.split("/")[0]
        word = word.encode('utf-8')
        adjTerms.append(word)
        adjectives[domain]=adjTerms




In [13]:
adjectiveDF = pd.DataFrame.from_dict(adjectives,orient='index')



adjectiveDF = adjectiveDF.reset_index()
adjectiveDF = adjectiveDF.rename(columns={"index":"reviewID"})
adjectiveDF.head()



,reviewID,0,1,2,3,4,5,6,7
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None
2,app_4,पुराने,None,None,None,None,None,None,None
3,app_6,नए,मुश्किल,None,None,None,None,None,None
4,app_9,मुताबिक,कोर,None,None,None,None,None,None


In [14]:
domain=[]
for id in adjectiveDF.index:
    domain.append(adjectiveDF.iloc[id,0][:3])
adjectiveDF['domain'] = domain

adjectiveDF

,reviewID,0,1,2,3,4,5,6,7,domain
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None,app
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None,app
2,app_4,पुराने,None,None,None,None,None,None,None,app
3,app_6,नए,मुश्किल,None,None,None,None,None,None,app
4,app_9,मुताबिक,कोर,None,None,None,None,None,None,app
5,app_10,यूनिक,None,None,None,None,None,None,None,app
6,app_11,इन्स्पायर्ड,मॉडर्न,None,None,None,None,None,None,app
7,app_12,अडिशनल,None,None,None,None,None,None,None,app
8,app_13,खास,None,None,None,None,None,None,None,app
9,app_14,तहत,None,None,None,None,None,None,None,app


In [15]:
entAdjectives = entertainmentDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')
elecAdjectives = electronicDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')

In [16]:
entAdjectives = entAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)


In [17]:
elecAdjectives = elecAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)

In [18]:
entAdjectives.head(20)

,domainID,aspectCategoryPolarity,review,aspectCategory,0,1,2,3,4,5,6,7
0,mov_3,neu,जॉम्बी जोनर में बनने वाली फिल्म ग्लैमरस तो नही...,misc,बेस्ड,None,None,None,None,None,None,None
1,mov_9,con,फिल्म की कहानी अच्छी होते हुए भी उसे ठीक से पर...,story,अच्छी,None,None,None,None,None,None,None
2,mov_12,neg,बहुत ही डलनेस सा महसूस होता है।,misc,महसूस,None,None,None,None,None,None,None
3,mov_13,neu,अर्नाल्ड इस बार बहुत ही सीरियस किरदार में है।,performance,सीरियस,None,None,None,None,None,None,None
4,mov_16,con,अर्नाल्ड की मौजूदगी से फिल्म में कुछ देर के लि...,story,सही,None,None,None,None,None,None,None
5,mov_17,neu,फिल्म में कुछ देर बाद अन्य हॉरर फिल्मों जैसा ह...,misc,अन्य,महसूस,None,None,None,None,None,None
6,mov_20,neg,यही इस फिल्म की सबसे बड़ी कमजोरी है।,misc,बड़ी,None,None,None,None,None,None,None
7,mov_23,pos,"दोनों देशों के बीच किरदारों के मेल-मिलाप, हंसी...",misc,दोनों,हास्यास्पद,मौजूद,None,None,None,None,None
8,mov_25,con,"अफसोस कि 'वेलकम 2 कराची' का विचार तो नेक, कटाक...",misc,कमजोर,None,None,None,None,None,None,None
9,mov_28,neu,"अभी जो भूमिका जैकी भगनानी ने निभाई है, उसी भूम...",performance,जैकी,None,None,None,None,None,None,None


In [46]:
entLexicon={}
for id in entAdjectives.index:
    category = entAdjectives.iloc[id,3]
    adjectives = entAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    #print adjectives
    for items in adjectives:
        adjList=[]
        if category in entLexicon.keys():
            entLexicon[category].append(items)
        else:
            adjList.append(items)
            entLexicon[category]=adjList
            
elecLexicon={}
for id in elecAdjectives.index:
    category = elecAdjectives.iloc[id,3]
    adjectives = elecAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    for items in adjectives:
        adjList=[]
        if category in elecLexicon.keys():
            elecLexicon[category].append(items)
        else:
            adjList.append(items)
            elecLexicon[category] = adjList
    

In [49]:
for keys in elecLexicon.keys():
    print len(elecLexicon[keys])

301
629
2053
120
753
497


In [22]:
word = "खाना".decode('utf-8', 'ignore')

import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
#synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
#synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))


if word2Synset.has_key(word):
    synsets = word2Synset[word]
    for pos in synsets.keys():
        for synset in synsets[pos]:
            if synonyms.has_key(synset):
                print "\t\t\t Synonyms -->", synonyms[synset]
                
                
            


			 Synonyms --> {'1': [u'\u0916\u093e\u0926\u094d\u092f_\u0935\u0938\u094d\u0924\u0941', u'\u0916\u093e\u0926\u094d\u092f_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0916\u093e\u0926\u094d\u092f\u0935\u0938\u094d\u0924\u0941', u'\u0916\u093e\u0926\u094d\u092f\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0906\u0939\u093e\u0930', u'\u0916\u093e\u0926\u094d\u092f', u'\u092d\u094b\u091c\u094d\u092f_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0916\u093e\u0926\u094d\u092f_\u0938\u093e\u092e\u0917\u094d\u0930\u0940', u'\u0916\u093e\u0926\u094d\u092f-\u0938\u093e\u092e\u0917\u094d\u0930\u0940', u'\u0906\u0939\u093e\u0930_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0905\u0928\u094d\u0928', u'\u0906\u0939\u0930', u'\u0916\u093e\u0928\u093e', u'\u092b\u0942\u0921', u'\u0930\u0938\u0926', u'\u0905\u0930\u094d\u0915', u'\u0907\u0930\u093e', u'\u0924\u0906\u092e']}
			 Synonyms --> {'1': [u'\u092d\u094b\u091c\u0928', u'\u0916\u093e\u0928\u093e', u'\u0906\u0939\u093e\u0930', u'\u0905\u0928\u094d\u0928', u

In [23]:
import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))

In [50]:
import pickle
word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
#synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
def getSynonyms(word):
    """import pickle

    word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
    synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
    synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))"""
    #synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
    #synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))

    synList=[]
    if word2Synset.has_key(word):
        synsets = word2Synset[word]
        for pos in synsets.keys():
            for synset in synsets[pos]:
                if synonyms.has_key(synset):
                    #print "\t\t\t Synonyms -->", synonyms[synset]
                    synDict = synonyms[synset]
                    synList.append(synDict)
    #print len(synList)
    return synList
"""word = "खाना".decode('utf-8', 'ignore')
synList = getSynonyms(word)
for key,value in synList[1].iteritems():
    for item in value:
        print item"""
        
for key,item in entLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        entLexicon[key].append(syns)
       
        """for k,value in synList[1].iteritems():
            for syns in value:
                lexicon[key].append(syns)"""
        
for key,item in elecLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        elecLexicon[key].append(syns)


In [52]:
for keys in elecLexicon.keys():
    print len(elecLexicon[keys])

2461
5439
14753
830
6839
3513


In [53]:
import json,ast
with open("Z:/MSBooks/NLP/Project/termextracted/aspect_terms_output.txt") as termFile:
    termData = json.load(termFile)
termData = ast.literal_eval(json.dumps(termData, ensure_ascii=False, encoding='utf8'))


In [54]:
from collections import OrderedDict
categoryDict=OrderedDict()
for key,items in termData.iteritems():
    for k,v in items.iteritems():
        if len(v)>0:
            for items in v:
                for lexKey,lexItems in entLexicon.iteritems():
                    if items in lexItems:
                        categoryDict[key] = lexKey
                for lk,li in elecLexicon.iteritems():
                    if i in li:
                        categoryDict[key] = lk

C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [55]:
categoryDict

OrderedDict([('tab_768', u'performance'),
             ('tab_764', u'performance'),
             ('mob_1329', u'performance'),
             ('mob_1327', u'performance'),
             ('mob_1325', u'performance'),
             ('app_108', u'performance'),
             ('app_35', u'misc'),
             ('cam_88', u'place'),
             ('cam_86', u'misc'),
             ('cam_84', u'performance'),
             ('mov_871', u'performance'),
             ('mov_873', u'performance'),
             ('mov_876', u'performance'),
             ('mov_879', u'place'),
             ('sma_124', u'performance'),
             ('sma_125', u'scenery'),
             ('tab_378', u'misc'),
             ('cam_102', u'performance'),
             ('cam_103', u'performance'),
             ('cam_100', u'performance'),
             ('tab_575', u'performance'),
             ('mob_852', u'music'),
             ('mob_429', u'performance'),
             ('mob_856', u'music'),
             ('tab_1228', u'misc'),
      

In [57]:
#predictedCategoryDF = pd.DataFrame.from_dict(categoryDict)
predictedCategoryDF = pd.DataFrame(categoryDict.items(), columns=['reviewID', 'predictedCategory'])
predictedCategoryDF.head()

,reviewID,predictedCategory
0,tab_768,performance
1,tab_764,performance
2,mob_1329,performance
3,mob_1327,performance
4,mob_1325,performance


In [58]:
finalDF = categoryDF.merge(predictedCategoryDF,left_on="domainID",right_on="reviewID")
finalDF

,domainID,aspectCategoryPolarity,review,aspectCategory,domain,reviewID,predictedCategory
0,app_4,neu,आप यहीं से पुराने लॉगइन सेशंस को हमेशा के लिए ...,gui,app,app_4,place
1,app_6,pos,अब नए फीचर से इस तरह की गलतियां नहीं होंगी और ...,NaN,app,app_6,performance
2,app_17,pos,आपके स्मार्टफोन में ज्यादातर एप ऎसे ही हैं जिन...,price,app,app_17,place
3,app_21,pos,3जी पर मैसेज डिलीवरी व्हाट्सप्प और आर्इमैसेज क...,ease_of_use,app,app_21,misc
4,app_29,pos,हालांकि इसके अधिकतर भाग सही स्टैंडर्ड रूप में ...,NaN,app,app_29,performance
5,app_35,pos,इसके टेक्स्ट आधारित चैट का अनुभव ठीक वैसा ही ह...,ease_of_use,app,app_35,misc
6,app_47,pos,"ऐसी सर्विस शुरू करने वाला, यह तीसरा डायरेक्ट-ट...",misc,app,app_47,performance
7,app_50,con,"इसका एप्प इंटरफेस बहुत रोमांचक तो नहीं है, लेक...",gui,app,app_50,place
8,app_52,pos,लाल और सफेद का कलर थीम कम्बिनेशन एयरटेल का बहु...,NaN,app,app_52,performance
9,app_55,pos,मौजूदा डिजाइन निस्संदेह उपयोग और प्रेरणा से अध...,misc,app,app_55,music


In [63]:
finalDF = finalDF.dropna()

In [65]:
from sklearn.metrics import f1_score
f1_score(finalDF['aspectCategory'], finalDF['predictedCategory'], average='micro')  

0.23159018143009605

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(finalDF['aspectCategory'], finalDF['predictedCategory'])  

0.23159018143009605